In [1]:
import PublicDataReader as pdr
from config import OpenAPI

print(pdr.__all__)
print(pdr.__version__)

['__version__', '__info__', 'Transaction', 'Building']
2021.11.


In [2]:
# 법정동코드 목록
code = pdr.code_list()
code.head()

,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,NaN,1100000000,NaN,NaN,19880423,NaN
1,11,서울특별시,11110,종로구,1111000000,NaN,NaN,19880423,NaN
2,11,서울특별시,11110,종로구,1111010100,청운동,NaN,19880423,NaN
3,11,서울특별시,11110,종로구,1111010200,신교동,NaN,19880423,NaN
4,11,서울특별시,11110,종로구,1111010300,궁정동,NaN,19880423,NaN


In [3]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Transaction(serviceKey, debug=True)

[INFO] 아파트매매 실거래자료 조회 - (00) NORMAL SERVICE.
[INFO] 아파트매매 실거래 상세 자료 조회 - (00) NORMAL SERVICE.
[INFO] 아파트 전월세 자료 조회 - (00) NORMAL SERVICE.
[INFO] 아파트 분양권전매 신고 자료 조회 - (00) NORMAL SERVICE.
[INFO] 오피스텔 매매 신고 조회 - (00) NORMAL SERVICE.
[INFO] 오피스텔 전월세 신고 조회 - (00) NORMAL SERVICE.
[INFO] 연립다세대 매매 실거래자료 조회 - (00) NORMAL SERVICE.
[INFO] 연립다세대 전월세 실거래자료 조회 - (00) NORMAL SERVICE.
[INFO] 단독/다가구 매매 실거래 조회 - (00) NORMAL SERVICE.
[INFO] 단독/다가구 전월세 자료 조회 - (00) NORMAL SERVICE.
[INFO] 토지 매매 신고 조회 - (00) NORMAL SERVICE.
[INFO] 상업업무용 부동산 매매 신고 자료 조회 - (00) NORMAL SERVICE.


In [4]:
prod = "아파트"
trans = "매매"
sigunguCode = "11680"
yearMonth = "202110"

df = self.read_data(prod, trans, sigunguCode, yearMonth)
df

,거래금액,거래유형,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,...,월,일,일련번호,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"41,000",,2001,2021,언주로86길,00011,00000,11680,01,0,...,10,1,11680-249,41.68,,706-20,11680,24,,
1,"45,000",,2004,2021,강남대로84길,00033,00000,11680,01,0,...,10,2,11680-668,43.99,,824-25,11680,10,,
2,"298,000",,2001,2021,언주로86길,00012,00000,11680,01,0,...,10,5,11680-252,149.7,,710,11680,3,,
3,"20,500",,2012,2021,역삼로25길,00027,00000,11680,01,0,...,10,6,11680-4036,17.684,,727-2,11680,2,,
4,"39,000",,2001,2021,언주로86길,00011,00000,11680,01,0,...,10,8,11680-249,42.52,,706-20,11680,20,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,"279,000",,1983,2021,논현로,00218,00000,11680,05,0,...,10,9,11680-332,153.71,,869,11680,12,,
192,"410,000",,2006,2021,선릉로,00221,00000,11680,08,0,...,10,9,11680-3623,114.9931,,527,11680,6,,
193,"220,000",,1985,2021,언주로,00123,00000,11680,05,0,...,10,15,11680-349,52.73,,464,11680,3,,
194,"210,000",,1986,2021,언주로29길,00034,00000,11680,01,0,...,10,16,11680-338,70.89,,463-1,11680,4,,


In [5]:
self.metaDict['아파트']['매매']['columns']

['거래금액',
 '거래유형',
 '건축년도',
 '년',
 '도로명',
 '도로명건물본번호코드',
 '도로명건물부번호코드',
 '도로명시군구코드',
 '도로명일련번호코드',
 '도로명지상지하코드',
 '도로명코드',
 '법정동',
 '법정동본번코드',
 '법정동부번코드',
 '법정동시군구코드',
 '법정동읍면동코드',
 '법정동지번코드',
 '아파트',
 '월',
 '일',
 '일련번호',
 '전용면적',
 '중개사소재지',
 '지번',
 '지역코드',
 '층',
 '해제사유발생일',
 '해제여부']

In [6]:
self.AptTrade(sigunguCode, yearMonth)

,지역코드,법정동,거래일,아파트,지번,전용면적,층,건축년도,거래금액
0,11680,개포동,2021-10-02,디에이치아너힐즈,138,94.2875,19,2019,340000
1,11680,개포동,2021-10-04,성원대치2단지아파트,12,49.8600,10,1992,175000
2,11680,개포동,2021-10-04,삼익대청아파트,12,39.5300,2,1992,149500
3,11680,개포동,2021-10-04,삼익대청아파트,12,39.5300,7,1992,144000
4,11680,개포동,2021-10-06,개포주공5단지,187,83.1700,1,1983,279500
...,...,...,...,...,...,...,...,...,...
191,11680,청담동,2021-10-01,청담자이,134-38,90.1970,14,2011,330000
192,11680,청담동,2021-10-02,범신칼릭스빌,125,84.7800,3,2005,150000
193,11680,청담동,2021-10-04,청담동삼성1차아파트,33,59.4000,9,1997,162500
194,11680,청담동,2021-10-13,청담동삼성1차아파트,33,84.5700,5,1997,192000


## 컬럼 리스트 추출

In [ ]:
# LAWD_CD=11680
# DEAL_YMD=202110


# for prod in ['아파트','오피스텔','단독다가구','연립다세대','상업업무용','분양입주권','토지']:
#     for trans in ['매매', '전월세']:
#         try:
#             print()
#             print(f">> {prod} {trans}")
#             print()
#             service_url = self.metaDict[prod][trans]['url']

#             # URL
#             url=f"""\
# {service_url}\
# &LAWD_CD={str(LAWD_CD)}\
# &DEAL_YMD={str(DEAL_YMD)}\
# &numOfRows=99999\
#             """

#             result = requests.get(url, verify=False)
#             xmlsoup = BeautifulSoup(result.text, "lxml-xml")
#             item = xmlsoup.find("item")

#             col_list = []
#             for i in item:
#                 col_list.append(i.name)
#             for col in col_list: 
#                 print(f"'{col}'", end=',')
                
#         except:
#             print("Error")
#             pass

## Develop

In [ ]:
import pandas as pd
import numpy as np
import datetime
import logging
import requests
from bs4 import BeautifulSoup

### Features

- 로깅 기능
    - 세션 초기화 시 `debug` 인자 추가
        - False(Default): 오류 레벨만 출력
        - True: 모든 레벨 출력
- OPEN API 에러 코드 별 조치방안 출력 기능 추가
    - 
- Operation 별 조회 항목 수정
    - 매매자료 '해제사유발생일' 항목 추가
    - 거래유형, 중개사소재지 항목 추가
- Air Korea Open API 모듈 제거
- 법정동코드 테이블 조회 함수 추가
- 데이터 조회 메서드 일원화
    - molit.read_data('아파트', '매매', '11680', '202111')
        - 서비스별 End-point 및 컬럼 매핑 딕셔너리 추가